In [254]:
import pandas as pd
from tqdm import tqdm
import gensim
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.chdir(r"D://Proposal_Defense//Simulations")
# from Utils.Script_utils import get_data_splits, first_LSTM_training, get_data_splits_old_algo
from joblib import Parallel, delayed
import sys
from scipy.stats import norm

In [255]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# # 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
# tf.config.set_visible_devices([], 'GPU')

from tqdm.notebook import tqdm
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# from keras import backend as K
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# K.set_session(sess)

No GPU found


In [256]:
import matplotlib.pyplot as plt

import pickle

In [257]:
# def get_parent_dir(directory):
#     import os
#     return os.path.dirname(directory)

# current_dirs_parent = get_parent_dir(os.getcwd())

In [258]:
# os.chdir(current_dirs_parent)

In [259]:
# current_dirs_parent

In [260]:
model_cbow = gensim.models.word2vec.Word2Vec.load(r"..//word2vec_sg")

In [261]:
lr = 1e-3
# from sklearn.model_selection import train_test_split


In [262]:
reps = 50

In [263]:
# with open('true_data_doc2vec_lstm_var_0.0001.pkl', 'wb') as f:
#     pickle.dump(catch, f)


with open('..//Data//train_valid_test_splits_50.pkl', 'rb') as f:
    catch = pickle.load(f)  

In [264]:
# rate = 0.5

In [265]:
doc2vec_dbow = gensim.models.doc2vec.Doc2Vec.load(r"..//doc2vec_dbow")

In [266]:
doc2vec_shape = doc2vec_dbow.dv.vectors.shape[1]

In [267]:
def doc2vec_ann(rate, training): 
    
    input_layer = tf.keras.layers.Input(shape = (doc2vec_shape))

    hidden_layer = tf.keras.layers.Dense(16, kernel_regularizer=tf.keras.regularizers.L1())
    
    hidden_output = hidden_layer(input_layer)
    
    drop_layer = tf.keras.layers.Dropout(rate)
    
    dropout_out = drop_layer(hidden_output, training = training)  

    pred_head = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L1())

    pred_output = pred_head(dropout_out)

    model = tf.keras.models.Model(input_layer, pred_output)
    
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                 optimizer = tf.keras.optimizers.Adam(learning_rate = lr), 
                 metrics=tf.keras.metrics.BinaryAccuracy())
    
    return model

In [268]:
samp_model = doc2vec_ann(0.75, False)

In [269]:
# samp_model.summary()

In [270]:
# chk = attention_lstm_model(rate, False)

In [271]:
# chk.layers

In [272]:
# rate = 0.5

In [275]:
def doc2vec_ann_training(idx):
    
    train_doc2vec = []
    for seq in catch[idx][0]:
        seq_txt = seq[0]
        seq_split = seq_txt.split(" ")
        train_doc2vec.append(doc2vec_dbow.infer_vector(seq_split))
    train_doc2vec = np.array(train_doc2vec)
    
    valid_doc2vec = []
    for seq in catch[idx][1]:
        seq_txt = seq[0]
        seq_split = seq_txt.split(" ")
        valid_doc2vec.append(doc2vec_dbow.infer_vector(seq_split))
    valid_doc2vec = np.array(valid_doc2vec)
    
    test_doc2vec = []
    for seq in catch[idx][2]:
        seq_txt = seq[0]
        seq_split = seq_txt.split(" ")
        test_doc2vec.append(doc2vec_dbow.infer_vector(seq_split))
    test_doc2vec = np.array(test_doc2vec)      

    doc2vec_ann_mod = doc2vec_ann(0.75, False)
    # model_word2vec = simple_lstm(False, model_cbow)
    # init_weights = first_lstm.get_weights()
    
    doc2vec_ann_mod.fit(train_doc2vec, catch[idx][3], epochs = 2000, verbose = 0, 
                  callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 30,
                                                              restore_best_weights=True), 
                      validation_data = (valid_doc2vec, catch[idx][4]))
    
    X_train_logits = doc2vec_ann_mod.predict(train_doc2vec, verbose = 0)
    X_valid_logits = doc2vec_ann_mod.predict(valid_doc2vec, verbose = 0)
    X_test_logits = doc2vec_ann_mod.predict(test_doc2vec, verbose = 0)
    
    doc2vec_ann_mod.save("Real_World_Doc2Vec_ANN" + "//" + "Model_" + str(idx))
    
    
    # print(doc2vec_ann_mod.evaluate(test_doc2vec, catch[idx][5]))
    
    return X_train_logits, X_valid_logits , X_test_logits

In [276]:
first_doc2vec_ann = Parallel(n_jobs=15, verbose = 10, backend = "loky")(delayed(doc2vec_ann_training)( i) for i in range(reps))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.


1/1 [==============================] - 0s 18ms/step - loss: 0.6016 - binary_accuracy: 0.7500


1/1 [==============================] - 0s 18ms/step - loss: 0.4605 - binary_accuracy: 0.8750


[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:   12.8s


1/1 [==============================] - 0s 19ms/step - loss: 0.4330 - binary_accuracy: 1.0000


...

[0.7098003029823303, 0.75]
1/1 [==============================] - 0s 18ms/step - loss: 0.3691 - binary_accuracy: 1.0000
[0.4306969940662384, 0.875]
1/1 [==============================] - 0s 17ms/step - loss: 0.4877 - binary_accuracy: 1.0000


[0.5566946268081665, 0.75]
1/1 [==============================] - 0s 17ms/step - loss: 0.9909 - binary_accuracy: 0.7500


[0.5667020082473755, 0.875]
1/1 [==============================] - 0s 17ms/step - loss: 0.2875 - binary_accuracy: 1.0000


[0.4648818373680115, 0.875]
1/1 [==============================] - 0s 17ms/step - loss: 0.5159 - binary_accuracy: 0.8750


[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:  2.0min finished


In [277]:
with open('doc2vec_ann.pkl', 'wb') as f:
    pickle.dump(first_doc2vec_ann, f)

[0.4801290035247803, 0.875]
[0.47867393493652344, 0.875]
[0.35818323493003845, 1.0]
[0.5603498220443726, 0.875]
[0.8599236607551575, 0.625]
[0.2065814882516861, 1.0]
[0.5891433358192444, 0.75]
[0.7701529860496521, 0.625]
[0.5280088186264038, 1.0]
[0.3069707453250885, 1.0]
[0.3691491186618805, 1.0]
[0.4877047538757324, 1.0]
[0.9909160733222961, 0.75]
[0.2874676585197449, 1.0]
[0.515872597694397, 0.875]


In [219]:
# catch_train_logits = []
# catch_train_probs = []
# catch_train_labels = []
# catch_train_word2vec = []
# catch_train_seqs = []
# # catch_train_word2vec = []
# for i in range(0, reps):
#     # train_true_probs = tf.math.sigmoid(first_lstm[i][0])
# #     idxes = ((train_true_probs >= 0.05) & (train_true_probs <= 0.95)).numpy().nonzero()[0]
# #     catch_train_idxes.append(idxes)
#     train_logits = first_doc2vec_ann[i][0]
#     catch_train_logits.append(train_logits)
#     # train_probs = train_true_probs.numpy()
#     # catch_train_probs.append(train_probs)
#     # train_labels = catch[i][3]
#     # catch_train_labels.append(train_labels)
#     # # catch_train_word2vec.append(catch[i][6])
#     # catch_train_seqs.append(catch[i][0])

In [220]:
# catch_valid_logits = []
# catch_valid_probs = []
# catch_valid_labels = []
# catch_valid_word2vec = []
# catch_valid_seqs = []
# # catch_train_word2vec = 
# for i in range(0, reps):
#     # valid_true_probs = tf.math.sigmoid(first_lstm[i][1])
#     valid_logits = first_doc2vec_ann[i][1]
#     catch_valid_logits.append(valid_logits)
#     # valid_probs = valid_true_probs.numpy()
#     # catch_valid_probs.append(valid_probs)
#     # valid_labels = catch[i][4]
#     # catch_valid_labels.append(valid_labels)
#     # # catch_valid_word2vec.append(catch[i][4])
#     # catch_valid_seqs.append(catch[i][1])

In [221]:
# catch_test_logits = []
# catch_test_probs = []
# catch_test_labels = []
# # catch_test_word2vec = []
# catch_test_seqs = []
# for i in range(0, reps):
#     # test_true_probs = tf.math.sigmoid(first_lstm[i][2])
# #     idxes = ((test_true_probs >= 0.05) & (test_true_probs <= 0.95)).numpy().nonzero()[0]
# #     catch_test_idxes.append(idxes)
#     test_logits = first_doc2vec_ann[i][2]
#     catch_test_logits.append(test_logits)
#     # test_probs = test_true_probs.numpy()
#     # catch_test_probs.append(test_probs)
#     # test_labels = catch[i][5]
#     # catch_test_labels.append(test_labels)
#     # # catch_test_word2vec.append(catch[i][5])
#     # catch_test_seqs.append(catch[i][2])

In [222]:
# with open('doc2vec_ann_train_logits.pkl', 'wb') as f:
#     pickle.dump(catch_train_logits, f)

In [223]:
# with open('doc2vec_ann_valid_logits.pkl', 'wb') as f:
#     pickle.dump(catch_valid_logits, f)

In [224]:
# with open('doc2vec_ann_test_logits.pkl', 'wb') as f:
#     pickle.dump(catch_test_logits, f)

In [225]:
# ens_model = tf.keras.models.load_model("Real_World_LSTM_Models" + "//" + "Model_" + str(0))

In [226]:
# ens_model.summary()

In [227]:
# base_model = attention_lstm_model(True)

In [228]:
# base_model.set_weights(ens_model.get_weights())

In [229]:
# base_model.predict(catch[0][0])